# Training a Classifier on the *Salammbô* Dataset with Keras
Pierre Nugues

We first need to import some modules

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import numpy as np

### Reading the dataset
We can read the data from a file with the svmlight format or directly create numpy arrays

In [ ]:
y_train = np.array(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

X_train = np.array(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ],dtype=np.float32)

## Scaling the Data
Scaling and normalizing are usually very significant with neural networks. We use sklean transformers. They consist of two main methods: `fit()` and `transform()`.

### Normalizing

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_train_norm

### Standardizing

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True,with_std=True)
scaler.fit(X_train_norm)
X_train_scaled = scaler.transform(X_train_norm)
X_train_scaled

## Fitting the Data

We set a seeed to have reproducible results

In [ ]:
np.random.seed(1337)

We create a classifier and fit a model

In [ ]:
model = Sequential()
model.add(Dense(1, input_dim=2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=300, batch_size=1)

### The weights

In [ ]:
model.get_weights()

## We evaluate the model

In [ ]:
pTest = model.predict(X_train_scaled, batch_size=1)
pTest

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def predict(preds):
    c = []
    for x in range(len(preds)):
        if(preds[x] >= 0.5):
            c += [1]
        else:
            c += [0]
    return np.array(c)

In [ ]:
cla = predict(pTest)
cla

In [ ]:
accuracy_score(y_train, cla)